In [1]:
import langchain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import textwrap
import simple_llm_cache
import llm_cache_stats_wrapper
import os

# In order to make it easy to run work projects and personal AI experiments, override OPENAI_API_KEY with the value of OPENAI_API_KEY_PERSONAL if it is set.
if "OPENAI_API_KEY_PERSONAL" in os.environ:
    print("Using key from OPENAI_API_KEY_PERSONAL environment variable")
    os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY_PERSONAL"]

verbose = False
temperature = 0.5

langchain.llm_cache = llm_cache_stats_wrapper.LlmCacheStatsWrapper(simple_llm_cache.SimpleLlmCache("llm-cache.json"))

def dump_cache_stats_since_last_call():
    print(langchain.llm_cache.get_cache_stats_summary())
    langchain.llm_cache.clear_cache_stats()

template = """Answer the following question as if you are a {character} character:
{question}
"""

prompt = prompt = PromptTemplate(
    input_variables=["character", "question"],
    template=template)

for model_name in ["gpt-3.5-turbo", "gpt-4", "gpt-4o"]:
    llm = ChatOpenAI(
        temperature=temperature,
        model_name = model_name)

    chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=verbose)

    for trial in range(2):
        print(f"\n*** {model_name} trial {trial} ***")
        langchain.llm_cache.inner_cache.set_trial(trial)
        output = chain.predict(
            character="pirate",
            question="What is a neural network?")
        print(textwrap.fill(output, width=80))

print()
dump_cache_stats_since_last_call()

c:\Users\forre\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\forre\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(



*** gpt-3.5-turbo trial 0 ***
Arrr, ye scurvy landlubber! A neural network be like a crew of brainy buccaneers
workin' together to solve problems and learn from their experiences. They be
usin' algorithms and data to make decisions and predict outcomes, just like me
trusty parrot helpin' me find me buried treasure! Aye, a neural network be a
powerful tool for navigatin' the high seas of information and plunderin'
valuable insights. So hoist the Jolly Roger and set sail for knowledge, me
hearties!

*** gpt-3.5-turbo trial 1 ***
Arr, me matey, a neural network be like a ship's crew of brainy cells workin'
together to learn and make decisions. It be a fancy way o' teachin' a computer
to think like a human, makin' it smarter and more powerful than ever before. Ye
see, just like me and me crew rely on each other to sail the high seas, a neural
network relies on its interconnected nodes to process information and solve
problems. It be a powerful tool in the world o' technology, helpin' us p